In [3]:
from google.colab import files
uploaded = files.upload()


In [ ]:
!pip install gradio --quiet
import gradio as gr


In [ ]:
print(X.columns.tolist())


['age', 'workclass', 'fnlwgt', 'education', 'educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']


In [ ]:
columns_list = X.columns.tolist()

def predict_salary(age, education_num, hours_per_week, workclass, occupation, sex):
    # Create input data in same order
    input_data = pd.DataFrame([[
        age,
        education_num,
        hours_per_week,
        workclass,
        occupation,
        sex
    ]], columns=columns_list)

    prediction = model.predict(input_data)[0]
    return f"Predicted Salary: ₹{round(prediction, 2)}"


In [ ]:
print(df.columns.tolist())


['age', 'workclass', 'fnlwgt', 'education', 'educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import gradio as gr

# Load the dataset
df = pd.read_csv("adult 3.csv")

# Strip whitespaces in column names
df.columns = df.columns.str.strip()

# Drop rows with missing values
df = df.dropna()

# Save original labels
education_map = df[['education', 'educational-num']].drop_duplicates().sort_values('educational-num')
workclass_options = sorted(df['workclass'].unique())
occupation_options = sorted(df['occupation'].unique())
gender_options = sorted(df['gender'].unique())

# Label encoding
le_work = LabelEncoder()
le_occ = LabelEncoder()
le_gen = LabelEncoder()
le_income = LabelEncoder()

df['workclass'] = le_work.fit_transform(df['workclass'])
df['occupation'] = le_occ.fit_transform(df['occupation'])
df['gender'] = le_gen.fit_transform(df['gender'])
df['income'] = le_income.fit_transform(df['income'])

# Select features and target
X = df[['age', 'educational-num', 'hours-per-week', 'workclass', 'occupation', 'gender']]
y = df['income']

# Train model
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Prediction function
def predict_income(age, edu_label, hours, workclass_label, occupation_label, gender_label):
    try:
        workclass = le_work.transform([workclass_label])[0]
        occupation = le_occ.transform([occupation_label])[0]
        gender = le_gen.transform([gender_label])[0]
        edu_num = education_map.loc[education_map['education'] == edu_label, 'educational-num'].values[0]

        data = pd.DataFrame([[age, edu_num, hours, workclass, occupation, gender]],
                            columns=X.columns)

        prediction = model.predict(data)[0]
        return le_income.inverse_transform([prediction])[0]
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio UI
iface = gr.Interface(
    fn=predict_income,
    inputs=[
        gr.Number(label="Age"),
        gr.Dropdown(choices=education_map['education'].tolist(), label="Education Level"),
        gr.Number(label="Hours per Week"),
        gr.Dropdown(choices=workclass_options, label="Workclass"),
        gr.Dropdown(choices=occupation_options, label="Occupation"),
        gr.Dropdown(choices=gender_options, label="Gender"),
    ],
    outputs="text",
    title="Income Prediction Web App",
    description="Predict whether a person earns >50K or <=50K using demographic info."
)

iface.launch(share=True)


FileNotFoundError: [Errno 2] No such file or directory: 'adult 3.csv'